# Listing all topologies for a finite set

Consider a a finte $n$-elements set 
$$X=\{0,1,\ldots,n-1\}.$$
We can encode a subset $S \subset 2^X$ as a binary tuple ("binary mask") of length $n$: 
$$S\simeq(b_0, b_1,\ldots, b_n),\quad b_i\in\{0,1\},$$
and $b_i=1$ whenever $i\in S$.

For three subsets $S^j\simeq(b^j_i)_i$, $j=1,2,3$ we have
\begin{equation}
    \begin{aligned}
    S^3 &= S^1 \cap S^2 \Leftrightarrow b^3_i=b_i^1 b_i^2,\\
    S^3 &= S^1 \cup S^2 \Leftrightarrow b^3_i=b_i^1 + b_i^2 - b_i^1 b_i^2.
    \end{aligned}
\end{equation}

A set of tuples $T\in  2^{\{0,1\}^n}\simeq 2^{2^X}$ can be considered as providing topology for $X$, whenever
1. $(1,\ldots,1)\in T$ and  $(0,\ldots,0)\in T$ ($X$ and $\emptyset$ are inside);
2. For any $b^1, b^2\in T$: $(b^1+b^2-b^1b^2)\in T$ (this correponds to a union of any two subsets);
3. For any $b^1, b^2\in T$: $b^1b^2\in T$ (this correponds to an intersection of any two subsets).

All operations with tuples are considered in a "point-wise" fashion, like in Eq.(1). Note that the second point is valid due to finitness of $X$, otherwise one should consider arbitrary uniouns.

Next, we make a code to list all the topologies for $n$-elements sets via a exhaustive check over all $2^{2^{n}}$ candidates.

![title](_not_funny_memes/slaboumie_i_otvaga.png)

In [1]:
def join_subsets(subset1: tuple, subset2: tuple) -> tuple:
    """Makes a union of two subsets represented as tuples"""
    assert len(subset1) == len(subset2)
    assert (
        max(subset1) <= 1
        and max(subset2) <= 1
        and min(subset1) >= 0
        and min(subset2) >= 0
    )
    set_rslt = []
    for a, b in zip(subset1, subset2):
        set_rslt.append(a + b - a * b)
    return tuple(set_rslt)


def intersect_subsets(subset1: tuple, subset2: tuple) -> tuple:
    """Makes an itersection of two subsets represented as tuples"""
    assert len(subset1) == len(subset2)
    assert (
        max(subset1) <= 1
        and max(subset2) <= 1
        and min(subset1) >= 0
        and min(subset2) >= 0
    )
    set_rslt = []
    for a, b in zip(subset1, subset2):
        set_rslt.append(a * b)
    return tuple(set_rslt)


def make_set_of_subsets_from_bin_mask(bin_mask: list, n: int) -> set:
    """Reconstructs a set of subsets (tuples) from a binary tuple of 2^n length 
    for given n.
    For example, for n=2, (1,0,0,1) is converted to the antidiscrete topology:
    1 (0,0) empty set is inside
    0 (0,1) {1} is not included
    0 (1,0) {0} is not included
    1 (1,1) full set {0,1} is inside
    """
    assert len(bin_mask) == 2**n
    set_of_subsets = set()
    for ind in range(2**n):
        if bin_mask[ind] == 1:
            subset = tuple([int(dit) for dit in bin(ind)[2:].zfill(n)])
            set_of_subsets.add(subset)
    return set_of_subsets


def make_bin_mask_from_int(ind: int, n: int) -> tuple[int]:
    """Prepares a 'binary mask' of length 2^n from an 'index' of this mask 
    (from 0 to 2^(2^n)-1). Goes as input for 
    make_set_of_subsets_from_bin_mask(bin_mask, n)
    Example: for n = 2
    ind = 0 -> (0, 0, 0, 0)
    ind = 1 -> (0, 0, 0, 1)
    ind = 2 -> (0, 0, 1, 0)
    ...
    ind = 15 -> (1, 1, 1, 1)
    """
    assert 0 <= ind < 2 ** (2**n)
    bin_mask = tuple([int(dit) for dit in bin(ind)[2:].zfill(2**n)])
    return bin_mask


def convert_subset_to_familiar_form(subset: tuple) -> tuple:
    """Converts subset in the form of a binary tuple to a tuple with 
    weight(sebset) elements with particular elements.
    Example: (1, 0, 1, 1, 0) is converted to frozenset {0, 2, 4}.
    """
    assert max(subset) <= 1 and min(subset) >= 0
    n = len(subset)
    set_rslt = []
    for i in range(n):
        if subset[i] == 1:
            set_rslt.append(i)
    return frozenset(set_rslt)
    #return tuple(set_rslt)


def convert_set_of_subsets_to_topology(
    set_of_subsets: set[tuple],
) -> set[tuple]:
    """Converts sets-of-subsets in the form like {(0, 0, 0), (1, 1, 1)}
    to a more readable form {(), (0, 1, 2)}"""
    topology = set()
    for subset in set_of_subsets:
        topology.add(convert_subset_to_familiar_form(subset))
    return topology

In [2]:
# *** Playground ***
n = 2
ind = 10
print(
    f"{ind=} for {n=} corresponds to a mask of "
    f"a subset of set's subsets'{make_bin_mask_from_int(ind, n)}"
)

bin_mask = (1, 1, 0, 1)
print(
    f"For a mask {bin_mask}  and {n=} we have "
    f"set-of-subsets: {make_set_of_subsets_from_bin_mask(bin_mask, n=2)}"
)

subset = (1, 1)
print(
    f"Subset encoded in {subset} is actually "
    f"{set(convert_subset_to_familiar_form(subset))}"
)

ind=10 for n=2 corresponds to a mask of a subset of set's subsets'(1, 0, 1, 0)
For a mask (1, 1, 0, 1)  and n=2 we have set-of-subsets: {(0, 1), (1, 1), (0, 0)}
Subset encoded in (1, 1) is actually {0, 1}


In [3]:
def check_topolginess(set_of_subsets: set, n: int = 3, verbose: bool = False) -> bool:
    """Checks whether a given set-of-subsets provides a topology for given n"""
    # Check that the whole set is in the set_of_subsets
    if tuple([1] * n) not in set_of_subsets:
        if verbose:
            print(
                "ERROR: "
                f"Full set {tuple([1] * n)} is not in the given set of subsets"
            )
        return False

    # Check that the empty set is in the set_of_subsets
    if tuple([0] * n) not in set_of_subsets:
        if verbose:
            print(
                f"ERROR: Empty set "
                f"{tuple([1] * n)} is not in the given set of subsets"
            )
        return False

    for ind, subset1 in enumerate(set_of_subsets):
        for ind, subset2 in list(enumerate(set_of_subsets))[ind + 1 :]:
            # Check that union is in the set_of_subsets
            if join_subsets(subset1, subset2) not in set_of_subsets:
                if verbose:
                    print(
                        f"ERROR:: Join of {subset1} and {subset2} is not in "
                        "the given set of subsets"
                    )
                return False
            # Check that intersection is in the set_of_subsets
            if intersect_subsets(subset1, subset2) not in set_of_subsets:
                if verbose:
                    print(
                        f"ERROR: Intersection of {subset1} and {subset2} "
                        "is not in  the given set of subsets"
                    )
                return False
    return True

In [4]:
# *** Playground ***
# Sanite check for the antidiscrete topology
set_of_subsets = {(1, 1, 1), (0, 0, 0)}
print(check_topolginess(set_of_subsets, 3))

# Sanite check for some arbitrary set of subsets
set_of_subsets = {(1, 1, 1), (0, 1, 0)}
print(check_topolginess(set_of_subsets, 3, verbose=True))

True
ERROR: Empty set (1, 1, 1) is not in the given set of subsets
False


In [5]:
def print_topology(topology: set) -> None:
    """Printing topology in a pretty way"""
    N = len(topology)
    output = "{"
    for i, subset in enumerate(topology):
        subset_str = str(set(subset)) if len(subset) > 0 else '{}'
        output += subset_str
        if i < N - 1:
            output += ', '
        else:
            output += '}'
    print(output)

In [6]:
n = 3

list_of_topologies = []
for ind in range(2 ** (2**n)):
    bin_mask = make_bin_mask_from_int(ind, n)
    set_of_subsets = make_set_of_subsets_from_bin_mask(bin_mask, n=n)
    if check_topolginess(set_of_subsets, n=n, verbose=False):
        topology = convert_set_of_subsets_to_topology(set_of_subsets)
        list_of_topologies.append(topology)
print(f"For {n=}, {len(list_of_topologies)} topologies in total")
print(f"Explicit list of all topologies for a set {set(range(n))}:")
for topology in list_of_topologies:
    print_topology(topology)

For n=3, 29 topologies in total
Explicit list of all topologies for a set {0, 1, 2}:
{{}, {0, 1, 2}}
{{}, {0, 1}, {0, 1, 2}}
{{}, {0, 2}, {0, 1, 2}}
{{}, {0, 1, 2}, {0}}
{{}, {0, 1}, {0, 1, 2}, {0}}
{{}, {0, 1, 2}, {0, 2}, {0}}
{{0, 1, 2}, {0, 1}, {0, 2}, {}, {0}}
{{}, {1, 2}, {0, 1, 2}}
{{}, {0, 1, 2}, {1, 2}, {0}}
{{}, {1}, {0, 1, 2}}
{{}, {0, 1}, {1}, {0, 1, 2}}
{{}, {0, 2}, {1}, {0, 1, 2}}
{{0, 1, 2}, {0, 1}, {1}, {}, {0}}
{{0, 1, 2}, {0, 1}, {0, 2}, {1}, {}, {0}}
{{}, {1}, {1, 2}, {0, 1, 2}}
{{1, 2}, {0, 1, 2}, {0, 1}, {1}, {}}
{{0, 1, 2}, {1, 2}, {0, 1}, {1}, {}, {0}}
{{}, {2}, {0, 1, 2}}
{{}, {2}, {0, 1}, {0, 1, 2}}
{{}, {2}, {0, 2}, {0, 1, 2}}
{{2}, {0, 1, 2}, {0, 2}, {}, {0}}
{{2}, {0, 1, 2}, {0, 1}, {0, 2}, {}, {0}}
{{}, {2}, {1, 2}, {0, 1, 2}}
{{2}, {1, 2}, {0, 1, 2}, {0, 2}, {}}
{{2}, {0, 1, 2}, {1, 2}, {0, 2}, {}, {0}}
{{2}, {1, 2}, {0, 1, 2}, {1}, {}}
{{2}, {0, 1, 2}, {1, 2}, {0, 1}, {1}, {}}
{{2}, {0, 1, 2}, {1, 2}, {0, 2}, {1}, {}}
{{2}, {0, 1, 2}, {1, 2}, {0, 1}, {0, 2

In [7]:
# Going once again and storing all found topologies for given n
topologies_dict = {}
for n in [1, 2, 3, 4]:
    list_of_topologies = []
    for ind in range(2 ** (2**n)):
        bin_mask = make_bin_mask_from_int(ind, n)
        set_of_subsets = make_set_of_subsets_from_bin_mask(bin_mask, n=n)
        if check_topolginess(set_of_subsets, n=n, verbose=False):
            topology = convert_set_of_subsets_to_topology(set_of_subsets)
            list_of_topologies.append(topology)
    topologies_dict[n] = list_of_topologies  # keeping generated topologies

## Studying separation axioms

Here, we will examine whether a given topological space $(X, T)$ satisfies one or more of the standard separation axioms that are commonly used in topology.

* The $T_0$ axiom states that all points in $X$ are topologically distinguishable, meaning that no two points in X have the same neighbourhood.

* The $T_1$ axiom requires that for any two distinct points $x$ and $y$ in $X$, there exist corresponding neighbourhoods $U_x$ and $U_y$ such that $x\not\in U_y$ and $y\not\in U_x$.

* The $T_2$ axiom requires that for any two distinct points $x$ and $y$ in $X$, there exist corresponding neighbourhoods $U_x$ and $U_y$ such that $U_x \cap U_y=\emptyset$ (more strict than $T_1$).

* The $T_3$ axiom requires that for any closed set $C$ and point $x\not\in C$, there exist corresponding neighbourhoods $U_C$ and $U_x$ such that $U_C \cap U_x=\emptyset$.

* The $T_4$ axiom requires that for any nonintersecting closed sets $C$ and $D$, there exist corresponding neighbourhoods $U_C$ and $U_D$ such that $U_C \cap U_D=\emptyset$.

* If a topological space $(X, T)$ satisfies both the $T_1$ and the $T_3$ conditions, it is called *regular*.

* Similarly, if $(X, T)$ satisfies the $T_1$ and $T4$ conditions, then it is said to be *normal*.

One can check that normality $\Rightarrow$ regularity $\Rightarrow$ $T_2$ (but not vice versa).

In [8]:
from itertools import product


def get_closed_sets(topology: set[frozenset], n: int) -> set[frozenset]:
    """Generates all closed sets out of open sets for topology on a set of n
    elements"""
    full_set = frozenset(range(n))
    closed_sets = set()
    for open_set in topology:
        closed_set_cur = full_set - open_set
        closed_sets.add(closed_set_cur)
    return closed_sets


def get_neighbourhoods_for_point(
    topology: set[frozenset], x: int, n: int
) -> set[frozenset]:
    """Generates all neighbourhoods for x within topology on a set of n
    elements"""
    assert 0 <= x < n
    neighbourhoods = set()
    for open_set in topology:
        if x in open_set:
            neighbourhoods.add(open_set)
    return neighbourhoods


def get_neighbourhoods_for_set(
    topology: set[frozenset], subset: frozenset, n: int
) -> set[frozenset]:
    """Generates all neighbourhoods for x within topology on a set of n
    elements"""
    neighbourhoods = set()
    for open_set in topology:
        if subset.issubset(open_set):
            neighbourhoods.add(open_set)
    return neighbourhoods

In [9]:
def test_t0(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T0 separation axiom"""
    for x in range(n):
        V_x = get_neighbourhoods_for_point(topology, x, n)
        for y in range(x + 1, n):
            V_y = get_neighbourhoods_for_point(topology, y, n)
            if V_x == V_y:  # x and y are topologically indistiguishable
                if verbose:
                    print_topology(topology)
                    print(f"{x=} and {y=} have the same neighbourhoods :-(")
                return False
    return True


def test_t1(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T1 separation axiom"""
    for x in range(n):
        V_x = get_neighbourhoods_for_point(topology, x, n)
        for y in range(x + 1, n):
            neig_for_x_found = False
            for U in V_x:
                if not y in U:
                    neig_for_x_found = True
                    break
            if not neig_for_x_found:
                if verbose:
                    print_topology(topology)
                    print(f"For {x=} and {y=} no U in V_x without {x} found "
                          ":-(")
                return False
            V_y = get_neighbourhoods_for_point(topology, y, n)
            neig_for_y_found = False
            for U in V_y:
                if not x in U:
                    neig_for_y_found = True
                    break
            if not neig_for_y_found:
                if verbose:
                    print_topology(topology)
                    print(f"For {y=} and {x=} no U in V_y without {y} found "
                          ":-(")
                return False
    return True


def test_t2(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T2 separation axiom"""
    for x in range(n):
        neigh_x = get_neighbourhoods_for_point(topology, x, n)
        for y in range(x + 1, n):
            neigh_y = get_neighbourhoods_for_point(topology, y, n)
            # Starting a check for current x and y
            separate_neigh_found = False  
            for A in neigh_x:
                if separate_neigh_found:
                    break
                for B in neigh_y:
                    if not A.intersection(B):
                        # Non-intersectiog neighbourhoods found!
                        separate_neigh_found = True
                        break
            if not separate_neigh_found:
                if verbose:
                    print_topology(topology)
                    print(f"{x=} and {y=} do not have not-intersecting "
                          "neighbourhoods")
                return False
    return True


def test_t3(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T3 separation axiom"""
    closed_sets = get_closed_sets(topology, n)
    for x in range(n):
        neigh_x = get_neighbourhoods_for_point(topology, x, n)
        for C in closed_sets:
            if x in C:
                continue  # only non intersecting x and V are interesting
            neigh_C = get_neighbourhoods_for_set(topology, C, n)
            separate_neigh_found = False
            # Starting a check for current x and C
            for A, B in product(neigh_x, neigh_C):
                if not A.intersection(B):
                    # Non-intersecting neigh-s found
                    separate_neigh_found = True  
                    break
            if not separate_neigh_found:
                if verbose:
                    print_topology(topology)
                    print(
                        f"{x=} and C={set(C)} do not have not-intersecting "
                        "neighbourhoods"
                    )
                return False
    return True


def test_t3_(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T3 separation axiom"""
    closed_sets = get_closed_sets(topology, n)
    for x in range(n):
        neigh_x = get_neighbourhoods_for_point(topology, x, n)
        for C in closed_sets:
            if x in C:
                continue  # only non intersecting x and V are interesting
            neigh_C = get_neighbourhoods_for_set(topology, C, n)
            # Starting a check for current x and C
            separate_neigh_found = False  

            for A in neigh_x:
                if separate_neigh_found:
                    break
                for B in neigh_C:
                    if not A.intersection(B):
                        # Non-intersectiog neighbourhoods found!
                        separate_neigh_found = True
                        break
            if not separate_neigh_found:
                if verbose:
                    print_topology(topology)
                    print(
                        f"{x=} and C={set(C)} do not have not-intersecting "
                        "neighbourhoods"
                    )
                return False
    return True


def test_t4(topology: set[frozenset], n: int, verbose=False) -> bool:
    """Checks that topology satisfies T4 separation axiom"""
    closed_sets = get_closed_sets(topology, n)
    # Not optimal but readable cycle through all pairs
    for C, D in product(closed_sets, closed_sets):
        if C.intersection(D):
            continue  # only non-intersecting C and D are interesting
        neigh_C = get_neighbourhoods_for_set(topology, C, n)
        neigh_D = get_neighbourhoods_for_set(topology, D, n)
        separate_neigh_found = False
        for A, B in product(neigh_C, neigh_D):
            if not A.intersection(B):
                # Non-intersectiog neighbourhoods found!
                separate_neigh_found = True
                break
        if not separate_neigh_found:
            if verbose:
                print_topology(topology)
                print(
                    f"C={set(C)} and D={set(D)} do not have not-intersecting"
                    "neighbourhoods"
                )
            return False
    return True

In [10]:
# Two auxiliarly functions for nice printing


def pretty_bool(B: bool) -> str:
    """Turns bool into a nice symbol"""
    return "V" if B else "X"


def print_strings_of_bools(bool_list: list[bool]) -> str:
    """Makes a pretty string for a list of bools"""
    S = ""
    for B in bool_list:
        S = S + pretty_bool(B) + "\t"
    return S

In [11]:
n = 3

topologies_list = topologies_dict[n]
t0_num = 0
t1_num = 0
t2_num = 0
t3_num = 0
t4_num = 0
tr_num = 0
tn_num = 0
twow_num = 0
print("i\tT0\tT1\tT2\tT3\tT4\tR\tN")
for i, topology in enumerate(topologies_list):
    t0 = test_t0(topology, n, verbose=False)
    if t0:
        t0_num += 1
    t1 = test_t1(topology, n, verbose=False)
    if t1:
        t1_num += 1
    t2 = test_t2(topology, n, verbose=False)
    if t2:
        t2_num += 1
    t3 = test_t3_(topology, n, verbose=False)
    if t3:
        t3_num += 1
    t4 = test_t4(topology, n, verbose=False)
    if t4:
        t4_num += 1
    if t1 and t3:
        tr_num += 1
    if t1 and t4:
        tn_num += 1
    print(
        f"{i}\t" + print_strings_of_bools([t0, t1, t2, t3, t4, t1 and t3, t1 and t4]),
        end=" ",
    )

    # --- Handling interesting cases ---
    if t0 and not t1 and t3 and not t4:
        print("(!!)")
        twow_num += 1
    else:
        print("")

print(
    f"Total: {len(topologies_list)}\nT0: {t0_num}\n"
    f"T1: {t1_num}\nT2: {t2_num}\nT3: {t3_num}\nT4: {t4_num}\n"
    f" R: {tr_num}\n N: {tn_num}\n!!: {twow_num}"
)

i	T0	T1	T2	T3	T4	R	N
0	X	X	X	V	V	X	X	 
1	X	X	X	X	V	X	X	 
2	X	X	X	X	V	X	X	 
3	X	X	X	X	V	X	X	 
4	V	X	X	X	V	X	X	 
5	V	X	X	X	V	X	X	 
6	V	X	X	X	X	X	X	 
7	X	X	X	X	V	X	X	 
8	X	X	X	V	V	X	X	 
9	X	X	X	X	V	X	X	 
10	V	X	X	X	V	X	X	 
11	X	X	X	V	V	X	X	 
12	V	X	X	X	V	X	X	 
13	V	X	X	X	V	X	X	 
14	V	X	X	X	V	X	X	 
15	V	X	X	X	X	X	X	 
16	V	X	X	X	V	X	X	 
17	X	X	X	X	V	X	X	 
18	X	X	X	V	V	X	X	 
19	V	X	X	X	V	X	X	 
20	V	X	X	X	V	X	X	 
21	V	X	X	X	V	X	X	 
22	V	X	X	X	V	X	X	 
23	V	X	X	X	X	X	X	 
24	V	X	X	X	V	X	X	 
25	V	X	X	X	V	X	X	 
26	V	X	X	X	V	X	X	 
27	V	X	X	X	V	X	X	 
28	V	V	V	V	V	V	V	 
Total: 29
T0: 19
T1: 1
T2: 1
T3: 5
T4: 26
 R: 1
 N: 1
!!: 0


In [12]:
# *** Playground ***
topology = topologies_list[1]
#print_topology(topology)
test_t3(topology, n, verbose=True)

{{}, {0, 1}, {0, 1, 2}}
x=0 and C={2} do not have not-intersecting neighbourhoods


False